# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: a370e573-f07d-435b-ad73-004687cf2f68
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
Waiting for session a370e573-f07d-435b-ad73-004687cf2f68 to get into ready status...
Session a370e573-f07d-435b-ad73-004687cf2f68 has been created.



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [6]:
university_ranking_source = glueContext.create_dynamic_frame.from_catalog(database="demo_db",table_name="university_ranking_csvcsv",transformation_ctx="UniversityRanking_Source")

In [7]:
rankings_df = university_ranking_source.toDF() 
rankings_df.show(5)

+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|          university|year|rank_display|score|                link|       country|     city|       region|                logo|   type|research_output|student_faculty_ratio|international_students|size|faculty_count|
+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|Massachusetts Ins...|2017|           1|100.0|https://www.topun...| United States|Cambridge|North America|https://www.topun...|Private|      Very High|                  4.0|                 3,730|   M|        3,065|
| Stanford University|2017|           2| 98.7|https://www.topun...| United States| Stanford|North America|https://www.topun...|Private| 

In [8]:
rankings_df.printSchema()

root
 |-- university: string (nullable = true)
 |-- year: long (nullable = true)
 |-- rank_display: string (nullable = true)
 |-- score: double (nullable = true)
 |-- link: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- logo: string (nullable = true)
 |-- type: string (nullable = true)
 |-- research_output: string (nullable = true)
 |-- student_faculty_ratio: double (nullable = true)
 |-- international_students: string (nullable = true)
 |-- size: string (nullable = true)
 |-- faculty_count: string (nullable = true)


In [9]:
rankings_df.createOrReplaceTempView("university_ranking")

In [10]:
spark.sql("""SELECT university,year,rank_display,region,country FROM university_ranking""").show(5) 

+--------------------+----+------------+-------------+--------------+
|          university|year|rank_display|       region|       country|
+--------------------+----+------------+-------------+--------------+
|Massachusetts Ins...|2017|           1|North America| United States|
| Stanford University|2017|           2|North America| United States|
|  Harvard University|2017|           3|North America| United States|
|University of Cam...|2017|           4|       Europe|United Kingdom|
|California Instit...|2017|           5|North America| United States|
+--------------------+----+------------+-------------+--------------+
only showing top 5 rows


In [11]:
spark.sql("""SELECT university,year,rank_display, int(rank_display) as n_rank,region,country 
FROM university_ranking""").show(5)

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
+--------------------+----+------------+------+-------------+--------------+
only showing top 5 rows


In [12]:
spark.sql("""SELECT university,year,rank_display, int(rank_display) as n_rank,region,country 
FROM university_ranking""").show(5)

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
+--------------------+----+------------+------+-------------+--------------+
only showing top 5 rows


In [13]:
spark.sql("""SELECT university,year,rank_display, int(rank_display) as n_rank,region,country 
FROM university_ranking
WHERE rank_display like '%-%'""").show(5)

+--------------------+----+------------+------+-------------+--------------------+
|          university|year|rank_display|n_rank|       region|             country|
+--------------------+----+------------+------+-------------+--------------------+
|Bandung Institute...|2017|     401-410|  null|         Asia|           Indonesia|
| Brandeis University|2017|     401-410|  null|North America|       United States|
|Illinois Institut...|2017|     401-410|  null|North America|       United States|
|Johannes Gutenber...|2017|     401-410|  null|       Europe|             Germany|
|Khalifa Universit...|2017|     401-410|  null|         Asia|United Arab Emirates|
+--------------------+----+------------+------+-------------+--------------------+
only showing top 5 rows


In [14]:
spark.sql("""SELECT university,year,rank_display, int(split(rank_display,'-')[0]) as n_rank,region,country 
FROM university_ranking
WHERE rank_display like '%-%'""").show(5)

+--------------------+----+------------+------+-------------+--------------------+
|          university|year|rank_display|n_rank|       region|             country|
+--------------------+----+------------+------+-------------+--------------------+
|Bandung Institute...|2017|     401-410|   401|         Asia|           Indonesia|
| Brandeis University|2017|     401-410|   401|North America|       United States|
|Illinois Institut...|2017|     401-410|   401|North America|       United States|
|Johannes Gutenber...|2017|     401-410|   401|       Europe|             Germany|
|Khalifa Universit...|2017|     401-410|   401|         Asia|United Arab Emirates|
+--------------------+----+------------+------+-------------+--------------------+
only showing top 5 rows


In [15]:
spark.sql("""SELECT 
    university,
    coalesce(int(year),9999) as year,
    rank_display,
    coalesce(int(split(rank_display,'-')[0]),9999) as n_rank,
    coalesce(float(score),-1) as score,
    country, city, region, type,
    research_output,
    coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio,
    coalesce(int(regexp_replace(international_students,'[.,]','')),-1) as international_students,
    size,
    coalesce(int(regexp_replace(faculty_count,'[.,]','')),-1) as faculty_count
    FROM university_ranking""").show(5) 

+--------------------+----+------------+------+-----+--------------+---------+-------------+-------+---------------+---------------------+----------------------+----+-------------+
|          university|year|rank_display|n_rank|score|       country|     city|       region|   type|research_output|student_faculty_ratio|international_students|size|faculty_count|
+--------------------+----+------------+------+-----+--------------+---------+-------------+-------+---------------+---------------------+----------------------+----+-------------+
|Massachusetts Ins...|2017|           1|     1|100.0| United States|Cambridge|North America|Private|      Very High|                  4.0|                  3730|   M|         3065|
| Stanford University|2017|           2|     2| 98.7| United States| Stanford|North America|Private|      Very High|                  3.0|                  3879|   L|         4725|
|  Harvard University|2017|           3|     3| 98.3| United States|Cambridge|North America|Pri

In [16]:
clean_rankings_df = spark.sql("""SELECT 
    university,
    coalesce(int(year),9999) as year,
    rank_display,
    coalesce(int(split(rank_display,'-')[0]),9999) as n_rank,
    coalesce(float(score),-1) as score,
    country, city, region, type,
    research_output,
    coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio,
    coalesce(int(regexp_replace(international_students,'[.,]','')),-1) as international_students,
    size,
    coalesce(int(regexp_replace(faculty_count,'[.,]','')),-1) as faculty_count
    FROM university_ranking""") 

In [21]:
clean_dynamic_frame = DynamicFrame.fromDF(clean_rankings_df, glueContext, "university_ranking_clean") 

In [24]:

"""
import boto3
import pandas as pd
from io import StringIO


s3 = boto3.client('s3')

clean_df = clean_dynamic_frame.toDF()
csv_buffer = StringIO()
pd.to_csv(csv_buffer, index=False)
        
# Upload the CSV file
s3.put_object(Body=csv_buffer.getvalue(), Bucket='qsranking-rayon', Key='university_ranking/csv_clean/university_ranking_clean.csv')

"""

AttributeError: 'DataFrame' object has no attribute 'to_csv'


In [25]:
import boto3
import pandas as pd
from io import StringIO

# Convert DynamicFrame to DataFrame
clean_df = clean_dynamic_frame.toDF()

# Define the S3 bucket and file path
bucket_name = 'qsranking-rayon'
object_key = 'university_ranking/csv_clean/university_ranking_clean.csv'

# Save DataFrame to S3 as CSV
clean_df.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save("s3://" + bucket_name + "/" + object_key)

In [26]:
spark.stop()